measuring time to load data to GPU

In [1]:
import zlib
import numpy as np
from datasets import load_dataset
import torch

In [2]:
def entropy_decoder(compressed_img,original_shape):
    decompressed = zlib.decompress(compressed_img)
    ẑ = np.frombuffer(decompressed, dtype=np.int8)
    ẑ = ẑ.reshape(original_shape)
    return ẑ
def batch_entropy_decode(compressed_batch,latent_size):
    batch_size = latent_size[0];
    original_size = latent_size.numpy().copy(); original_size[0] = 1;
    z = torch.cat([torch.tensor(entropy_decoder(compressed_batch[i],original_size)) for i in range(batch_size)])
    return z

In [3]:
dataset = load_dataset("danjacobellis/imagenet_RDAE_batched_250k",split='train').with_format("torch")

In [4]:
%%time
for (i_batch, batch) in enumerate(dataset):
    latent_size = batch['latent_size']
    label = batch['label']
    compressed_batch = batch['compressed_batch']
    z = batch_entropy_decode(compressed_batch,latent_size)
    z.to("cuda")
    z.detach()

CPU times: user 10min 28s, sys: 14.2 s, total: 10min 42s
Wall time: 2min 33s
